# Coleta de Dados do Sentinel-2 usando Google Earth Engine

Este notebook demonstra como coletar dados do Sentinel-2 para áreas de interesse definidas em arquivos KML.

In [1]:
import sys
sys.path.append('..')

import ee
from pathlib import Path
from src.data.gee_collector import SentinelCollector
import geopandas as gpd

## 1. Configurar o Ambiente

Primeiro, vamos configurar os diretórios e inicializar os componentes necessários:

In [2]:
# Configurar diretórios
data_dir = Path('../data')
kml_dir = data_dir / 'kmls'  # Diretório para todos os KMLs

# Criar diretório se não existir
kml_dir.mkdir(parents=True, exist_ok=True)

# Inicializar o coletor
collector = SentinelCollector(train_val_dir=str(kml_dir))

In [3]:
gdf = gpd.read_file('/home/luisantolin/Downloads/GO_SOJA_2021/GO_SOJA_2021.shp',driver = 'ESRI')

gdf = gdf.sample(100, random_state =55)

gdf['name'] = [str(row.ID)+'-'+str(row.SIGLA_UF) for idx, row in gdf.iterrows()]
# Definir parâmetros da coleta
export_folder = 'sentinel_exports_gdf'

collector = SentinelCollector(
        train_val_dir='data/train_val_kmls'
)

/home/luisantolin/code/agri_remote_classification/.rmt/lib/python3.10/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver ESRI Shapefile does not support open option DRIVER
  return ogr_read(


In [6]:
start_date, end_date = collector._get_date_range()

In [ ]:
export_folder = 'sentinel_exports_gdf'

# Coletar dados
collector.collect_from_geodataframe(
        gdf=gdf,
        start_date=start_date,
        end_date=end_date,
        export_folder=export_folder,
        patch_size=64,
        scale=10,
        max_cloud_cover=20.0
    )

Starting collection from GeoDataFrame with 100 geometries...
Processing geometry: 13566-GO

Preparing to export 52 images for 13566-GO...
Started export task for 13566-GO_2020-04-27_region_0
Started export task for 13566-GO_2020-05-02_region_0
Started export task for 13566-GO_2020-06-21_region_0
Started export task for 13566-GO_2020-07-31_region_0
Started export task for 13566-GO_2020-08-25_region_0
Started export task for 13566-GO_2020-09-04_region_0
Started export task for 13566-GO_2020-10-04_region_0
Started export task for 13566-GO_2020-11-28_region_0
Started export task for 13566-GO_2020-12-03_region_0
Started export task for 13566-GO_2021-03-28_region_0
Started export task for 13566-GO_2021-04-22_region_0
Started export task for 13566-GO_2021-05-27_region_0
Started export task for 13566-GO_2021-06-26_region_0
Started export task for 13566-GO_2021-07-21_region_0
Started export task for 13566-GO_2021-08-15_region_0
Started export task for 13566-GO_2021-09-19_region_0
Started export

In [77]:
# Configurar parâmetros de exportação
export_params = {
    'scale': 10,  # Resolução de 10 metros
    'bands': ['B8', 'B4', 'B11'],  # Bandas relevantes para vegetação
    'folder': 'sentinel2_series'  # Pasta no Google Drive
}

In [78]:
# Processar cada KML independentemente
train_val_data = collector.dataset_manager.load_train_val_data()

Loaded 1 geometries from 48eeab1b-80f3-47e8-9418-97b136cea40f.kml
Loaded 1 geometries from 76b7ded7-6473-43cf-ae43-d6e594ca4604.kml
Loaded 1 geometries from 0f88127e-56c6-48f5-a423-61ba45393929.kml
Loaded 1 geometries from cec5e8a0-dcd7-4105-924b-1e189548c2f4.kml
Loaded 1 geometries from 3cb074f5-0c91-4bfc-90d6-b33b04043bf5.kml
Loaded 1 geometries from 10b9cefb-4123-4825-9988-e16723103b1c.kml
Loaded 1 geometries from 3173049e-e93e-40b6-b1b7-0e5744a217b2.kml
Loaded 1 geometries from 40074342-a7ee-494f-b0a9-672a1eaf9f8d.kml
Loaded 1 geometries from 2f139f8a-6405-4c8f-a054-4aa7c9354067.kml
Loaded 1 geometries from 2d428f83-1580-4f36-aa41-05ec7c7e5ddd.kml
Loaded 1 geometries from 45b8ee5e-5b51-49f8-b705-e977b3fafee9.kml
Loaded 1 geometries from b44b5051-0ad4-4e88-b77b-800733b5e379.kml
Loaded 1 geometries from 2a75477a-45a1-401a-8a5e-3e1dd009ae21.kml
Loaded 1 geometries from ac2fd63b-4b0e-4be5-953e-3d2dfad6b480.kml
Loaded 1 geometries from e3cdeb39-9b48-4a69-917e-2ebc841a6019.kml
Loaded 1 g

In [ ]:
for geometries, kml_name, kml_path in train_val_data:
    print(f"\nProcessando KML: {kml_name}")
    print(f"Número de geometrias: {len(geometries)}")
    
    # Criar collection específica para este KML
    collection = collector.get_sentinel_collection(
        regions=geometries,
        years_back=2,  # Últimos 2 anos de dados
        max_cloud_cover=20  # Máximo de 20% de cobertura de nuvens
    )
    
    # Exportar imagens para este KML
    collector.export_to_drive(
        collection=collection,
        regions=geometries,
        output_prefix=f"sentinel2_{kml_name}",
        **export_params
    )
    
    print(f"Exportação iniciada para {kml_name}")


Processando KML: 48eeab1b-80f3-47e8-9418-97b136cea40f
Número de geometrias: 1

Preparing to export 24 images for sentinel2_48eeab1b-80f3-47e8-9418-97b136cea40f...
Started export task for sentinel2_48eeab1b-80f3-47e8-9418-97b136cea40f_2023-04-10_region_0
Started export task for sentinel2_48eeab1b-80f3-47e8-9418-97b136cea40f_2023-05-25_region_0
Started export task for sentinel2_48eeab1b-80f3-47e8-9418-97b136cea40f_2023-06-04_region_0
Started export task for sentinel2_48eeab1b-80f3-47e8-9418-97b136cea40f_2023-07-21_region_0
Started export task for sentinel2_48eeab1b-80f3-47e8-9418-97b136cea40f_2023-08-03_region_0
Started export task for sentinel2_48eeab1b-80f3-47e8-9418-97b136cea40f_2023-09-22_region_0
Started export task for sentinel2_48eeab1b-80f3-47e8-9418-97b136cea40f_2023-10-07_region_0
Started export task for sentinel2_48eeab1b-80f3-47e8-9418-97b136cea40f_2023-11-08_region_0
Started export task for sentinel2_48eeab1b-80f3-47e8-9418-97b136cea40f_2023-12-28_region_0
Started export ta

## 2. Carregar Regiões

Vamos carregar todas as regiões dos KMLs:

In [3]:
# Carregar todas as regiões dos KMLs
regions, region_names = geometry_handler.load_kml_directory(kml_dir)
print(f"Número total de regiões: {len(regions)}")

Loaded 1 geometries from a8ec4540-3574-443d-99fb-107f4dcb42db.kml
Loaded 1 geometries from d2a4bf0d-45c7-4c17-b160-7dd440813d5c.kml
Loaded 1 geometries from b970c077-2956-47d7-98e3-9f4ee5cc5a54.kml
Loaded 1 geometries from c827e9cc-5b8a-4dfa-9c8a-630659550f1c.kml
Loaded 1 geometries from 99902fd9-66ed-463b-8e16-67f5f408bc94.kml
Loaded 1 geometries from 561564b9-a657-4b8e-bfee-585cf3400928.kml
Loaded 1 geometries from 2f48597d-7a88-4b9c-b464-f6ffd43db198.kml
Loaded 1 geometries from 4c64de73-22cd-4491-a2af-3139352c46a9.kml
Loaded 1 geometries from 2a98a92f-2e4d-469e-9fc4-5c165e52c870.kml
Loaded 1 geometries from 5e783152-5989-4959-8d3a-1a7d0507bf5a.kml
Loaded 1 geometries from 221495e5-d593-4b0f-b10e-85edaf5fab4e.kml
Loaded 1 geometries from ff45ecfe-5128-4db5-843c-f719686a4077.kml
Loaded 1 geometries from 339ea663-b9e5-4901-b5f4-84fb0650b55c.kml
Loaded 1 geometries from ca71104b-6515-421d-aa73-9d186e43c147.kml
Loaded 1 geometries from 386500de-a96a-41e2-aee0-ae1aac480927.kml
Loaded 1 g

## 3. Coletar Imagens do Sentinel-2

Agora vamos coletar todas as imagens válidas dos últimos 5 anos:

In [4]:
# Coletar imagens
collection = collector.get_sentinel_collection(
    regions=regions,
    years_back=2,  # Últimos 5 anos de dados
    max_cloud_cover=20  # Máximo de 20% de cobertura de nuvens
)

## 4. Exportar para Google Drive

Vamos exportar todas as imagens válidas para o Google Drive:

In [5]:
# Configurar parâmetros de exportação
export_params = {
    'scale': 10,  # Resolução de 10 metros
    'bands': ['B8', 'B4', 'B11'],  # Bandas relevantes para vegetação
    'folder': 'sentinel2_series',  # Pasta no Google Drive
    'region_names': region_names  # Nomes das regiões dos KMLs
}

# Exportar todas as imagens
collector.export_to_drive(
    collection=collection,
    regions=regions,
    output_prefix='sentinel2',
    **export_params
)


Preparing to export 17233 images for each of 633 regions...

Processing region: a8ec4540-3574-443d-99fb-107f4dcb42db (1/633)
Error exporting image 0 for region a8ec4540-3574-443d-99fb-107f4dcb42db: User memory limit exceeded.


KeyboardInterrupt: 

## 5. Monitoramento e Próximos Passos

### Monitoramento
As tarefas de exportação serão executadas no Google Earth Engine. Você pode monitorar o progresso em:
https://code.earthengine.google.com/tasks

### Organização dos KMLs
- Coloque todos os seus arquivos KML no diretório `kmls`
- O script irá processar todos os KMLs encontrados no diretório
- Você verá um resumo de quantas geometrias foram carregadas de cada arquivo

### Após a Exportação
1. As imagens serão exportadas para a pasta 'sentinel2_series' no seu Google Drive
2. Cada imagem terá o nome no formato: tiff/sentinel2_YYYY-MM-DD
3. As imagens serão exportadas no formato Cloud Optimized GeoTIFF

### Importante
Certifique-se de que você tem:
- Acesso configurado ao Google Earth Engine
- Espaço suficiente no Google Drive
- KMLs bem definidos para suas áreas de interesse

In [5]:
import pandas as pd
import glob
from pathlib import Path

In [48]:
# Find all KML files
tifs = glob.glob('/home/luisantolin/code/agri_remote_classification/data/tiffs/sentinel2_series/*.tif')
tifs = [i.split('_')[4] for i in tifs]

In [49]:
tifs = pd.DataFrame({'tifs_code':tifs},index=tifs)

In [50]:
tifs = tifs.groupby(level=0).count()
tifs

,tifs_code
0038c5ba-f969-435e-b1b1-78f99a025155,20
008ada01-9d70-47e1-bf72-d2fc4dae0488,23
0372647a-808c-4b8a-ad38-ca3c02608525,23
0470e20e-a860-465c-9148-ca79483d38d2,21
0a9c7d2e-f0b8-4792-8147-b5ffd514100f,24
...,...
f904002b-7e5d-42f9-bae8-f9672c6c261c,22
fb691bdf-e7f8-4ef8-99e9-451aeec8a88a,21
fe5de8f9-5719-43df-b341-f4b5c60f0bc1,23
ff38dbc4-e617-43b0-8328-ff854c2762bf,23


In [44]:
import shutil

In [54]:
src_path = '/home/luisantolin/code/agri_remote_classification/data/kmls/'

src_path = [src_path+i+'.kml' for i in tifs.index]

dst_path = '/home/luisantolin/code/agri_remote_classification/data/completed/'

dst_path = [dst_path+i+'.kml' for i in tifs.index]

In [69]:
src_path.remove('/home/luisantolin/code/agri_remote_classification/data/kmls/0372647a-808c-4b8a-ad38-ca3c02608525.kml')

ValueError: list.remove(x): x not in list

In [71]:
dst_path.remove('/home/luisantolin/code/agri_remote_classification/data/completed/0372647a-808c-4b8a-ad38-ca3c02608525.kml')

ValueError: list.remove(x): x not in list

In [74]:
for s,d in zip(src_path,dst_path):
    try:
        shutil.move(s, d)
    except:
        pass
    

In [62]:
s

'/home/luisantolin/code/agri_remote_classification/data/kmls/91402ee1-aee3-4e4b-bb57-b8cb7378969c.kml'